# SIMLAB1
## Adaptación del CLAB 21-22-Q1


### Control de errores 
Para todos los programas que se piden a continuación deben comprobarse los errores de TODAS las llamadas a sistema (excepto el write por pantalla), controlar los argumentos de entrada y definir la función **Usage()**.

### Makefile
Para todos los programas que se piden a continuación deben comprobarse los errores de TODAS las llamadas a sistema (excepto el write por pantalla). Para mostrar los errores y acabar la finalización por error de llamada a sistema, en el paquete junto al enunciado tenéis el **fichero error_exit.h** que contiene la especificación de la función que realiza esta tarea. Debéis invocarla en caso de que vuestro programa detecte errores en las llamadas a sistema. El código que implementa la función de **error_exit.h**  lo tenéis ya compilado en el fichero **error_exit.o**

Todos los programas también deben controlar los argumentos de entrada y definir la función **Usage()**.

In [ ]:
%%file Makefile
all: jerarquia1 jerarquia2

jerarquia1: jerarquia1.c error_exit.o
	gcc -o jerarquia1 jerarquia1.c error_exit.o
jerarquia2: jerarquia2.c error_exit.o
	gcc -o jerarquia2 jerarquia2.c error_exit.o

clean:
	rm -f jerarquia1 jerarquia2
	
    

### Ejercicio 1 
Escribe un programa (jerarquia1.c) que reciba una secuencia de entre 1 y 10 nombres de fichero como argumento. El programa debe crear de forma **concurrente**  tantos procesos hijo como nombres de fichero haya recibido. Cada proceso hijo se encarga de tratar uno de los ficheros. En este primer apartado el tratamiento consiste simplemente en mostrar por pantalla un mensaje con el nombre del fichero que le ha tocado y acabar la ejecución pasando como parámetro de finalización el número de orden de su creación. 

Por su parte, el proceso padre sólo tiene que esperar a que acaben todos los procesos hijos. La espera la tiene que hacer **en orden de creación** (hasta que no libere el PCB del primer proceso creado no puede pasar a liberar el PCB del segundo). Para cada hijo muerto, deberá mostrar en pantalla un mensaje con el pid del proceso hijo y con el parámetro que el hijo ha pasado al exit.

#### Programación

In [ ]:
%%file jerarquia1.c
#include <unistd.h>
#include <stdlib.h>
#include <string.h>
#include <stdio.h>
#include <errno.h>
#include <sys/wait.h>

#include "error_exit.h"

#define MAX_FILES 10

void Usage() {
    char buf[80];
    strcpy(buf,"Usage: jerarquia1 file1 [file2 .... file10]\n");
    write(1,buf,strlen(buf));
    exit(255);
}

int main(int argc, char *argv[]) {
int nhijos,i,status;
int pidh[10];
char buf[80];


if ((argc < 2) || (argc > MAX_FILES+1)) {
    Usage();
}

nhijos=argc-1;

for (i=0;i<nhijos;i++) {

    pidh[i]=fork();
    switch (pidh[i]) {
    case -1: error_exit("Error en creando hijos");
    case 0 :sprintf(buf,"Nombre de fichero: %s\n",argv[i+1]);
            write(1,buf,strlen(buf));
            exit(i);
    }

}

i=0;
while (waitpid(pidh[i],&status,0)>0) {
        if (WIFEXITED(status)) {
            sprintf(buf,"Pid del hijo muerto es %d, parametro del exit es %d\n", pidh[i],WEXITSTATUS(status));
            write(1,buf,strlen(buf));
        } 
        i++;
}


}


#### Compilación

In [ ]:
%%bash
make clean
make jerarquia1

#### Ejecución

In [ ]:
%%bash
./jerarquia1 f1 f2 f fd g 4 fg g gg ggg gggg

### Ejercicio 2 
Haz una copia de jerarquia1.c y llámala jerarquia2.c. Modifica el código de jerarquia2.c para que cada hijo haga ahora un tratamiento diferente. **Cada hijo**, después de mostrar el mensaje por pantalla con fichero que tiene que tratar, creará un grupo de tres procesos que se ejecutarán de manera **secuencial. Cada grupo de 3 “hermanos”** mutará de manera que ejecutarán la siguiente secuencia de comandos (siendo nombre_fichero el nombre del fichero que le toca tratar y nombre_nuevo se crea concatenando nombre_fichero y el sufijo “sin_repes”):

    1) wc -l nombre_fichero
Muestra por pantalla el número de líneas de “nombre_fichero”

    2) sort -u nombre_fichero -o nombre_nuevo
Guarda en el fichero “nombre_nuevo” el contenido ordenado de “nombre_fichero” y sin líneas repetidas

    3) wc -l nombre_nuevo
Muestra por pantalla el número de líneas de “nombre_nuevo”

#### Programación

In [ ]:
%%file jerarquia2.c
#include <unistd.h>
#include <stdlib.h>
#include <string.h>
#include <stdio.h>
#include <errno.h>
#include <sys/wait.h>
#include "error_exit.h"

#define MAX_FILES 10

void Usage() {
    char buf[80];
    strcpy(buf,"Usage: jerarquia1 file1 [file2 .... file10]\n");
    write(1,buf,strlen(buf));
    exit(1);
}


int main(int argc, char *argv[]) {
int nhijos,i,status,ret,j;
int pidh[10];
char buf[80];


if ((argc < 2) || (argc > MAX_FILES+1)) {
    Usage();
}

nhijos=argc-1;

for (i=0;i<nhijos;i++) {

    pidh[i]=fork();
    switch (pidh[i]) {
    case -1: error_exit("Error creando hijos primer nivel de jerarquia");
    case 0 :sprintf(buf,"Nombre de fichero: %s\n",argv[i+1]);
            write(1,buf,strlen(buf));
            
            for (j=0;j<3;j++){
                ret=fork();
                if (ret <0) error_exit("Error creando hijos segundo nivel de jerarquia");
                sprintf(buf,"%s_sin_repes",argv[i+1]); 
                if (ret==0) {
                    switch(j){
                        case 0: execlp("wc","wc","-l",argv[i+1],(char *) 0);
                                break;
                        case 1: execlp("sort","sort","-u",argv[i+1],"-o",buf,(char *) 0);
                                break;
                        case 2: execlp("wc","wc","-l",buf,(char *) 0);
                    }
                    error_exit("Error mutando a wc");
                }
                waitpid(-1,NULL,0);
             } 

            exit(i);
     }
}


i=0;
while (waitpid(pidh[i],&status,0)>0) {
        if (WIFEXITED(status)) {
            sprintf(buf,"Pid del hijo muerto es %d, parametro del exit es %d\n", pidh[i],WEXITSTATUS(status));
            write(1,buf,strlen(buf));
        } 
        i++;
}


}


#### Compilación

In [ ]:
%%bash
make clean
make jerarquia2

#### Ejecución

In [ ]:
%%bash
./jerarquia2 f1 f2 f3 f2_sin_repes

#### Prueba

In [ ]:
%%bash
cat f2

In [ ]:
%%bash
cat f2_sin_repes